# Fine Tuning a Pretrained Model
* Part 2: Using the `Trainer` class to fine tune pretrained models to a dataset

This Jupyter Notebook was not fully run due to the execution of the `Trainer` class being much faster using GPUs. A python equivalent and a slurm job have been included in the `runs` folder for the purpose of running the code displayed below.

In [14]:
import evaluate
import numpy as np

from datasets import load_dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments
)

In [11]:
# Summary of Chapter 3 so far
raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Found cached dataset glue (/n/fs/nlp-jy1682/hf_datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /n/fs/nlp-jy1682/hf_datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-2286bfeed35ec98c.arrow
Loading cached processed dataset at /n/fs/nlp-jy1682/hf_datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-d2871c4b8b165e62.arrow


  0%|          | 0/2 [00:00<?, ?ba/s]

Step 1: Define `TrainingArguments` class that contains all hyperparameters the `Trainer` will use for training + 
* Only required argument: Directory where trained model + checkpoints are saved

In [3]:
training_args = TrainingArguments("test-trainer")

/n/fs/nlp-jy1682/miniconda3/envs/learn/lib/python3.8/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Step 2: Define the model

In [5]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# Warning explained:
# - Reason: BERT has not been pretrained on classifying sentence pairs
# - State: Head of pretrained model is discarded, new head has been added
# - Warning: Some weights have not been used (the ones of the dropped pretrained head)
# - Recommendation: Train the model - What we're going to do!

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Step 3: Train the model. Pass in the model, training arguments, train + val datasets, data collator, and tokenizer

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"]
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer
)
trainer.train()

# This will start fine-tuning + report training loss every 500 steps
# This will *not* tell you anything about model performance
# - We did not set `evaluation_strategy` to either "steps" or "epoch"
# - We did not provide Trainer with a `compute_metrics()` function

Step 4: Add Evaluation
* Build a useful `compute_metrics()` function
* `compute_metrics()` should take an `EvalPrediction` object (named tuple w/ `predictions`, `label_ids` fields)
* `compute_metrics()` should return a dictionary mapping strings to floats
    * `string` is name of metric returned
    * `float` is the value of the metric

In [ ]:
# Get predictions from the model
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape) # (408, 2) (408,)

Output of `predict` explained:
* Tuple with three fields: `predictions`, `label-ids`, and `metrics`
* `metrics` includes loss + some time related things. This would also include `compute_metrics()` values
* `predictions` is 2D array with shape 408x2, where each row is logit
    * 408 = number of elements in validation set
* `label_ids` are ground truth labels

In [ ]:
# Transform logits into labels
preds = np.argmax(predictions.predictions, axis=-1)

# Compare predictions to real labels
metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

In [15]:
# Officially, we can write `compute_metrics` as follows:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Summary: New `Trainer` object with `compute_metrics` and additional training arguments

In [ ]:
# Tie it all together - new Trainer!
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()